labeling: tokenization problem 

XLNET: padding in the front

CrossentropyLoss: ignore padding

Prevent name tokenization errors: 
name = 123, 
if name12 already in output dont append,
if 

In [1]:
import pandas as pd
import numpy as np
import json
import time
import math
import re
import os
import torch
from transformers import  AdamW
from transformers import BertTokenizer, BertConfig, BertModel, BertPreTrainedModel, BertForTokenClassification
from transformers import XLNetTokenizer, XLNetConfig, XLNetModel, XLNetPreTrainedModel, XLNetForTokenClassification
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import get_linear_schedule_with_warmup
import ast
#from sklearn.cross_validation import train_test_split
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('stanford-corenlp-full-2018-10-05/', lang='zh')
from ckiptagger import WS, POS, NER

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# train_df = pd.read_csv('EuSun_train.csv')
# test_df = pd.read_csv('EuSun_valid.csv')
# print(len(train_df),len(test_df))
# train_df.head(2)

In [3]:
train_df = pd.read_csv('aug_updated_EuSun_train_0718.csv')
train_df.head(1)

,Unnamed: 0,content,full_content,hyperlink,link,name,news_ID,title
0,0,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易備受注目。專家表示，採用量化交易...,https://news.cnyes.com/news/id/4352432,NaN,[],1,NaN


In [4]:
train_df = pd.read_csv('final_train_0.9.csv')
test_df = pd.read_csv('final_valid_0.9.csv')
print(len(train_df),len(test_df))
train_df.head(3)

5886 683


,Unnamed: 0,Unnamed: 0.1,LINK,full_content,name,news_ID
0,0,0,https://mops.twse.com.tw/mops/web/t05st02?step...,1.董事會決議日期或發生變動日期:107/09/21\n\n2.人員別（請輸入董事長或總經理...,[],11
1,1,1,https://www.hk01.com/社會新聞/383851/逃犯條例-建造業議會指立法...,建造業議會指，2018至2019財政年度建造工程完成量按年下跌8.4%。主席陳家駒表示，本港...,[],12
2,2,2,https://news.mingpao.com/ins/%e6%b8%af%e8%81%9...,上午8時天文台錄得氣溫27度，相對濕度百分之75%。\n\n過去一小時，京士柏錄得的平均紫外...,[],20


In [5]:
test_df.head(1)

,Unnamed: 0,Unnamed: 0.1,LINK,full_content,name,news_ID
0,0,8,https://www.mirrormedia.mg/story/20191223web003/,馬雲21日在世界浙商上海論壇暨上海市浙江商會年會談話，談到做生意其實每一年日子都不容易，但是...,[],69


In [6]:
# cols_to_use = ['news_ID', 'hyperlink','name','full_content']
# new_data = pd.read_csv('new_data.csv', usecols= cols_to_use)
# new_data.head(3)
# frames = [train_df,new_data]
# train_df = pd.concat(frames,ignore_index=True)
# train_df.drop(train_df.columns[0], axis=1)
# print(len(train_df))
# train_df.head(3)

In [7]:
train_df['name'].to_list().count('[]')

4794

In [8]:
train_df['full_content'][3715]

'香港特首林鄭月娥週二 (9 日) 表示，擬議的逃犯條例法案已「壽終正寢」，修訂《逃犯條例》工作已徹底全面停止，希望香港市民不要因用不同字眼而有不同理解。\n\n林鄭月娥指出，社會的矛盾、紛爭、不滿及憤怒，都是因特區政府提出修訂《逃犯條例》而引起，政府今次修例工作完全失敗，包括對於社會脈搏掌握不夠，對政治敏感度有偏差，構成這次的失敗。\n\n過去數週以來，香港爆發了歷史性的遊行與抗議活動，林鄭曾一直強調修法的正常及必要性，拒絕撤回法案，也引來香港民眾一直存有疑慮，抗爭不斷。這次她仍未明言「撤回」，但確認法案已「壽終正寢」。\n\n林鄭指出，由這次的失敗，會加強行會成員收集意見的功能，包括行會成員應各自負責接觸不同群體，然後向特首反映各界意見。她還表示，願意與學生代表進行不設前提的公開對話。\n\n林鄭說，從 5 年前的佔中到今日反修例，都反映社會有很多深層次問題，政府不可以視而不見，應找出問題癥結，對症下藥，期盼可以修補社會撕裂。\n\n林鄭表示，她對香港人的素質感到自豪，絕大多數抗議者的和平行為表明了這一點。但是，她仍指責，極少數抗議者利用這種機會訴諸暴力及破壞，她很遺憾見到這些暴力行為，因為這破壞了香港的法治。\n\n她強調，要求大赦的想法，或是不調查及起訴示威期間違法的人，是「不可接受的」。\n\n'

In [9]:
train_df['name'][3715]

'[]'

In [10]:
#ckip
# from ckiptagger import data_utils
# data_utils.download_data_gdown("./")
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
ws = WS("./data", disable_cuda=False)
# pos = POS("./data", disable_cuda=False)
# ner = NER("./data", disable_cuda=False)
# # 18 entity types: https://github.com/ckiplab/ckiptagger/wiki/Entity-Types 
# # 0 for others, 1 for mlp names
# entity_dict = {'GPE':2,'PERSON':3,'DATE':4,'ORG':5,'CARDINAL':6,
# 'NORP':7,'LOC':8,'TIME':9,'FAC':10,'MONEY':11,'ORDINAL':12,'EVENT':13,
# 'WORK_OF_ART':14,'QUANTITY':15,'PERCENT':16,'LANGUAGE':17,'PRODUCT':18,'LAW':19}

In [11]:
class NewsDataset(Dataset):
    def __init__(self, mode, tokenizer, data):
        assert mode in ["train", "test"]  
        self.mode = mode
        self.data = data
        self.len = len(self.data)
        self.tokenizer = tokenizer  

    def __len__(self):
        return self.len
    
    def clean_text(self,text):
        text = text.replace('\r','')
        text = text.replace('\n','')
        return text
    
    def __getitem__(self, idx):
        MAX_LEN = 600
        content = self.data['full_content'][idx]
        text = self.clean_text(content)
        inputs = self.tokenizer.encode_plus(text=text, max_length=MAX_LEN, return_tensors='pt', 
                                            pad_to_max_length = True, 
                                            return_token_type_ids = True,
                                            return_attention_mask=True,
                                           truncation="longest_first",)
        
        input_ids = inputs['input_ids'].squeeze(0)
        segments_tensor = inputs['token_type_ids'].squeeze(0)
        masks_tensor = inputs['attention_mask'].squeeze(0)
        
        if self.mode == 'train':
            name_true = self.data['name'][idx] #list of 
            name_true = ast.literal_eval(name_true)
            tokens = self.tokenizer.tokenize(text)
            #print(tokens)
            labels = [4]*MAX_LEN
            if len(tokens) < MAX_LEN:
                PADDING_LEN = MAX_LEN - 2 - len(tokens)
            else:
                PADDING_LEN = 0
            
            if not name_true:
                return input_ids, torch.tensor(labels)
            else:
                #print(idx,name_true)
                for name in name_true:
                    #index = [i.start() for i in re.finditer(name, text)]
                    indexes = [i for i, e in enumerate(tokens) if e == name[0]]
                    for i in indexes:
                        if i < MAX_LEN-2-3:
                            if ''.join(tokens[i:i+len(name)]) == name:
                                for j in range(len(name)):
                                    if j == 0:
                                        labels[i+j+PADDING_LEN] = 1
                                    elif j == len(name)-1:
                                        labels[i+j+PADDING_LEN] = 3
                                    else:
                                        labels[i+j+PADDING_LEN] = 2
                return input_ids, torch.tensor(labels)
        else:
            newsid = self.data['news_ID'][idx]
            return newsid, input_ids


In [12]:
class BertForTokenClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = 4 ###
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, self.num_labels)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
        if labels is not None:
            #weight = [1-3631/(3893+3631), 1-(3893-3631)/(3893+3631)]
            weight = [1-0.8,1-0.2/3,1-0.2/3,1-0.2/3]
            class_weight = torch.FloatTensor(weight).to(device)
            loss_fct = torch.nn.CrossEntropyLoss()

            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
            else:
                #print(input_ids.shape,logits.shape,sequence_output.shape)
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), scores, (hidden_states), (attentions)


In [13]:
class XLNetForTokenClassification(XLNetPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = 5 ###
        self.transformer = XLNetModel(config)
        self.classifier = torch.nn.Linear(config.hidden_size, self.num_labels)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        mems=None,
        perm_mask=None,
        target_mapping=None,
        token_type_ids=None,
        input_mask=None,
        head_mask=None,
        inputs_embeds=None,
        use_cache=True,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):

        outputs = self.transformer(
            input_ids,
            attention_mask=attention_mask,
            mems=mems,
            perm_mask=perm_mask,
            target_mapping=target_mapping,
            token_type_ids=token_type_ids,
           
        )

        sequence_output = outputs[0]

        logits = self.classifier(sequence_output)
        
        outputs = (logits,) + outputs[1:]  # Keep mems, hidden states, attentions if there are in it
        if labels is not None:
            weight = [0.5,1-0.85,1-0.15/3,1-0.15/3,1-0.15/3]
            class_weight = torch.FloatTensor(weight).to(device)
            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=0,weight=class_weight)
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # return (loss), logits, (mems), (hidden states), (attentions)

In [14]:
def asMinutes(s): #s = time.time()-start_time
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def trainIter(MAX_LEN, PRETRAINED_MODEL_NAME, trainloader, valloader, epochs, LR):
    #config = BertConfig.from_pretrained(PRETRAINED_MODEL_NAME)
    #model = BertForTokenClassification.from_pretrained(PRETRAINED_MODEL_NAME)
    config = XLNetConfig.from_pretrained(PRETRAINED_MODEL_NAME)
    model = XLNetForTokenClassification.from_pretrained(PRETRAINED_MODEL_NAME)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.train().to(device)
    min_dev_loss, best_epoch = 1000000, 0
    start_time = time.time()
    optimizer = AdamW(model.parameters(),lr = LR)
    
    for epoch in range(epochs):
        total_steps = len(trainloader) * epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps = total_steps)
        
        steps, train_loss = 0, 0
        for data in trainloader:
            input_ids, labels = [t.to(device) for t in data] 
            outputs = model(input_ids, labels=labels)
            loss = outputs[0]
            loss.backward()
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            train_loss += loss.item()
            steps += 1
            print(f'Epoch : {epoch+1}/{epochs}, setps:{steps}, time: {asMinutes(time.time()-start_time)}, Training Loss : {train_loss/steps}',  end = '\r')
        print('\n===========================================================')
        
        prediction = evaluation(MAX_LEN, tokenizer, valloader,model)
        true_pred = [ast.literal_eval(i) for i in test_df['name']]
        name_pred = prediction['name_pred'].to_list()
        avg_f1, nonempty, actual_nonempty, no_names, actual_nonames, total_score = get_score(name_pred,true_pred)
        print(f'Epoch : {epoch+1}/{epochs}, time: {asMinutes(time.time()-start_time)}, Validation Score : {total_score}',  end = '\r')
        print('\n===========================================================')
        
        torch.save(model.state_dict(), f'xlnet_{int(total_score)}') 

In [15]:
# BS = 6
# PRETRAINED_MODEL_NAME = "bert-base-chinese"
# tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME, do_lower_case=True)

# # trainsize = int(0.9*len(train_df))
# # validsize = len(train_df) - trainsize
# # train_df, val_df = torch.utils.data.random_split(train_df,[trainsize, validsize])

# train = NewsDataset('train',tokenizer, train_df)
# trainloader = torch.utils.data.DataLoader(train, batch_size=BS,drop_last = True,shuffle=True)
# #val = NewsDataset('train',tokenizer, val_df)
# #valloader = torch.utils.data.DataLoader(val, batch_size=BS,drop_last = True,shuffle=True)

In [ ]:
BS = 6
PRETRAINED_MODEL_NAME = "hfl/chinese-xlnet-base"
tokenizer = XLNetTokenizer.from_pretrained(PRETRAINED_MODEL_NAME, do_lower_case=True)
train = NewsDataset('train',tokenizer, train_df)
trainloader = torch.utils.data.DataLoader(train, batch_size=BS,drop_last = True,shuffle=True)
test = NewsDataset('test',tokenizer, test_df)
testloader = torch.utils.data.DataLoader(test, batch_size=1,shuffle=False)

In [ ]:
# for data in testloader:
    
#     input_ids =  [i.to(device) for i in data[1]][0]
#     input_ids = input_ids.unsqueeze(0)
#     news_id = data[0]
#     print(news_id,input_ids.shape)
# #     input_ids, labels = [t.to(device) for t in data] 
# #     print(input_ids,labels.shape)
# # #     for l in labels:
# # #         print(l)
# #     #print(torch.nonzero(labels[0]))
#     break

In [ ]:
def evaluation(MAX_LEN, tokenizer, test_loader,model):
    prediction = pd.DataFrame(columns=['news_ID','name_pred'])
    fil = re.compile(u'[^0-9a-zA-Z\u4e00-\u9fa5.，,。？“”]+', re.UNICODE)
    with torch.no_grad():
        ids, pred = [], []
        for data in test_loader: 
            input_ids =  [i.to(device) for i in data[1]][0]
            input_ids = input_ids.unsqueeze(0)
            newsid = data[0]
            outputs = model(input_ids)
            ans_pred = torch.sigmoid(outputs[0])
            _, ans_pred = ans_pred.max(-1)
            #print(ans_pred)
            ans_pred = list(ans_pred[0])
            #ans_index = [i for i,x in enumerate(ans_pred) if x ==2]
            text = [tokenizer.convert_ids_to_tokens(i) for i in input_ids]
            text = list(text[0])
            
            PADDING_LEN = text.count('<pad>')
            ans_index = [i for i in range(PADDING_LEN+2,len(ans_pred)) if ans_pred[i] ==1]
            #print(ans_index)         
            ans_tokens = [text[x] for i,x in enumerate(ans_index)]
            #ans_tokens = [text[ans_index[i]]+text[ans_index[i]+1] for i in range(len(ans_index)-1)]
            
            ans_tokens=[]
            a = ''
            is_ans = False
            for i in range(len(ans_index)):
                index = ans_index[i]
                if ans_pred[index].item() == 1 and text[index] != '▁':
                    a = text[index]
                    for j in range(1,4):
                        #print(text[ans_index[i]],ans_index[i],ans_pred[ans_index[i]])
                        if ans_pred[index+j].item() == 3: 
                            a += text[index+j]
                            #print('1',a)
                            is_ans = True
                            break
                        else:
                            if index+j > MAX_LEN-3:
                                is_ans = False
                                break
                            else:
                                a += text[index+j]  
                if is_ans:
                    ans_tokens.append(fil.sub('', a))
                
            #ans_tokens = [text[ans_index[i]]+text[ans_index[i]+1]+text[ans_index[i]+2] for i in range(len(ans_index)-1)]
            
            names, answer = [], []
#             if ans_tokens != []:
#                 #print(ans_tokens)
#                 ws_results = ws([text])
#                 pos_results = pos(ws_results)
#                 ner_results = ner(ws_results, pos_results)
#                 #count word frequency
#                 word_dict = dict()
#                 for w in ws_results[0]:
#                     if w not in list(word_dict.keys()):
#                         word_dict[w] = 1 
#                     else:
#                         word_dict[w] += 1 
#                 for n in ner_results[0]:
#                     if n[2] == 'PERSON' and len(n[3]) < 5:
#                         names.append(n[3])
#             names = list(set(names))
#             names_2 = [x for x in names if len(x) == 2]
            #print(newsid,names)
            
            for t in ans_tokens:
                if t not in answer and len(t) < 5 and len(t) > 1:
                        if len(t) == 2 and t[1] in ['嫌','男','女','妻']:
                            print('discard',t)
                        elif len(t) == 4 and t[0]+t[1]+t[2] in names:
                            print('discard',t)
                        else:
                            answer.append(t)
                         
                        
            ids.append(newsid)
            pred.append(answer)
            print(f'news id : {newsid}, names : {answer}', end = '\r')
        prediction['news_ID'] = ids
        prediction['name_pred'] = pred
    return prediction
        
def get_score(name_preds, true_preds):
    total_score, no_names, actual_nonames = 0,0,0
    nonempty, actual_nonempty = 0,0
    total_f1 = 0
    for i in range(len(name_preds)):
        name_pred = name_preds[i]
        true_pred = true_preds[i]
        if true_pred == []:
            actual_nonames +=1
        else:
            actual_nonempty += 1
            
        if name_pred == [] and true_pred == []:
            total_score += 1
            no_names += 1
        elif name_pred != [] and true_pred != []:
            #print(name_pred,true_pred, set(name_pred) & set(true_pred))
            intersection = list(set(name_pred) & set(true_pred))
            recall = len(intersection)/len(true_pred)
            precision = len(intersection)/len(name_pred)
            if recall != 0 and precision != 0:
                f1 = 2/(1/recall+1/precision) 
            else:
                f1 = 0
            total_score += f1
            total_f1+=f1
            nonempty += 1
    return total_f1/nonempty, nonempty, actual_nonempty, no_names, actual_nonames, total_score

In [ ]:
trainIter(600, PRETRAINED_MODEL_NAME, trainloader, testloader, epochs=9, LR=5e-5)

In [23]:
def ckip_evaluation(MAX_LEN, tokenizer, test_loader,model):
    prediction = pd.DataFrame(columns=['news_ID','name_pred'])
    with torch.no_grad():
        ids, pred = [], []
        for data in test_loader: 
            input_ids =  [i.to(device) for i in data[1]][0]
            input_ids = input_ids.unsqueeze(0)
            newsid = data[0]
            outputs = model(input_ids)
            ans_pred = torch.sigmoid(outputs[0])
            _, ans_pred = ans_pred.max(-1)
            #print(ans_pred)
            ans_pred = list(ans_pred[0])
            #ans_index = [i for i,x in enumerate(ans_pred) if x ==2]
            ans_index = [i for i in range(len(ans_pred)) if ans_pred[i] ==1]
            #print(ans_index)
            text = [tokenizer.convert_ids_to_tokens(i) for i in input_ids]
            text = list(text[0])
            ans_tokens = [text[x] for i,x in enumerate(ans_index)]
            #ans_tokens = [text[ans_index[i]]+text[ans_index[i]+1] for i in range(len(ans_index)-1)]
            
            ans_tokens=[]
            a = ''
            is_ans = True
            for i in range(len(ans_index)):
                index = ans_index[i]
                if ans_pred[index].item() == 1:
                    a = text[index]
                
                    for j in range(1,4):
                        #print(text[ans_index[i]],ans_index[i],ans_pred[ans_index[i]])
                        if ans_pred[index+j].item() == 3: 
                            a += text[index+j]
                            #print('1',a)
                            break
                        else:
                            if index+j > MAX_LEN-3:
                                is_ans = False
                                break
                            else:
                                a += text[index+j]  
                if is_ans:
                    ans_tokens.append(a)
            
            
            
            names, answer = [], []       
            if ans_tokens != []:
                #print(ans_tokens)
                ws_results = ws([text])
                ckip_tokenized = list(set(ws_results[0]))
                
#             if ans_tokens != []:
#                 print(''.join(text).replace('<pad>',''))
#                 ner_results = nlp.ner(''.join(text).replace('<pad>',''))
#                 for n in ner_results[0]:
#                     if n[2] == 'PERSON' and len(n[3]) < 5:
#                         names.append(n[3])

            names = list(set(names))
            
            
            for t in ans_tokens:
                if t in ckip_tokenized and t not in answer and len(t) < 5 and len(t) > 1:
                    if len(t) == 2 and t[1] in ['嫌','男','女','妻']:
                        print('discard',t)
                    elif len(t) == 4 and t[0]+t[1]+t[2] in names:
                        print('discard',t)
                    else:
                        answer.append(t)
                         
                        
            ids.append(newsid)
            pred.append(answer)
            print(f'news id : {newsid}, names : {answer}', end = '\r')
        prediction['news_ID'] = ids
        prediction['name_pred'] = pred
    return prediction

In [28]:
def stanford_evaluation(MAX_LEN, tokenizer, test_loader,model):
    prediction = pd.DataFrame(columns=['news_ID','name_pred'])
    with torch.no_grad():
        ids, pred = [], []
        for data in test_loader: 
            input_ids =  [i.to(device) for i in data[1]][0]
            input_ids = input_ids.unsqueeze(0)
            newsid = data[0]
            outputs = model(input_ids)
            ans_pred = torch.sigmoid(outputs[0])
            _, ans_pred = ans_pred.max(-1)
            #print(ans_pred)
            ans_pred = list(ans_pred[0])
            #ans_index = [i for i,x in enumerate(ans_pred) if x ==2]
            ans_index = [i for i in range(len(ans_pred)) if ans_pred[i] ==1]
            #print(ans_index)
            text = [tokenizer.convert_ids_to_tokens(i) for i in input_ids]
            text = list(text[0])
            ans_tokens = [text[x] for i,x in enumerate(ans_index)]
            #ans_tokens = [text[ans_index[i]]+text[ans_index[i]+1] for i in range(len(ans_index)-1)]
            
            ans_tokens=[]
            a = ''
            is_ans = True
            for i in range(len(ans_index)):
                index = ans_index[i]
                if ans_pred[index].item() == 1:
                    a = text[index]
                
                    for j in range(1,4):
                        #print(text[ans_index[i]],ans_index[i],ans_pred[ans_index[i]])
                        if ans_pred[index+j].item() == 3: 
                            a += text[index+j]
                            #print('1',a)
                            break
                        else:
                            if index+j > MAX_LEN-3:
                                is_ans = False
                                break
                            else:
                                a += text[index+j]  
                if is_ans:
                    ans_tokens.append(a)
            
            names, answer = [], []       
            if ans_tokens != []:
                cleaned_text = ''.join(text).replace('<pad>','')
                ner_results = nlp.ner(cleaned_text)
                for n in ner_results:
                    if n[1] == 'PERSON' and len(n[0]) < 5:
                        names.append(n[0])
            names = list(set(names))
            
            
            for t in ans_tokens:
                if t in names and t not in answer and len(t) < 5 and len(t) > 1:
                    if len(t) == 2 and t[1] in ['嫌','男','女','妻']:
                        print('discard',t)
#                     elif len(t) == 4 and t[0]+t[1]+t[2] in names:
#                         print('discard',t)
                    else:
                        answer.append(t)
                         
                        
            ids.append(newsid)
            pred.append(answer)
            print(f'news id : {newsid}, names : {answer}', end = '\r')
        prediction['news_ID'] = ids
        prediction['name_pred'] = pred
    return prediction

In [81]:
# config = BertConfig.from_pretrained(PRETRAINED_MODEL_NAME)
# model = BertForTokenClassification(config)       
# model = model.cuda()
# checkpoint = torch.load('bert_2label_6_best')
# model.load_state_dict(checkpoint)
# model = model.eval()
# BS=1
# test = NewsDataset('test',tokenizer, test_df)
# testloader = torch.utils.data.DataLoader(test, batch_size=BS,shuffle=False)
# prediction = evaluation(tokenizer, testloader,model)

In [20]:
test_df = pd.read_csv('final_valid_0.9.csv')
config = XLNetConfig.from_pretrained(PRETRAINED_MODEL_NAME)
model = XLNetForTokenClassification(config)       
model = model.cuda()
checkpoint = torch.load('xlnet_660')
model.load_state_dict(checkpoint)
model = model.eval()
MAX_LEN = 600
BS=1
test = NewsDataset('test',tokenizer, test_df)
testloader = torch.utils.data.DataLoader(test, batch_size=BS,shuffle=False)
prediction = evaluation(MAX_LEN,tokenizer, testloader,model)

In [21]:
true_pred = [ast.literal_eval(i) for i in test_df['name']]
name_pred = prediction['name_pred'].to_list()
avg_f1, nonempty, actual_nonempty, no_names, actual_nonames, total_score = get_score(name_pred,true_pred)
print('empty name list:', no_names/actual_nonames, '====== (counts:',no_names, '/', actual_nonames)
print('non empty name list:', nonempty/actual_nonempty, '====== (counts:',nonempty, '/', actual_nonempty)
print('avg f1:', avg_f1)
print('total score:', total_score)

empty name list: 0.9852670349907919 ====== (counts: 535 / 543
non empty name list: 0.95 ====== (counts: 133 / 140
avg f1: 0.910305132918732
total score: 656.0705826781904


In [83]:
prediction['name_true'] = test_df['name']
prediction.to_csv('prediction_0720.csv')

In [61]:
true_pred = [ast.literal_eval(i) for i in test_df['name']]
name_pred = prediction['name_pred'].to_list()
avg_f1, nonempty, actual_nonempty, no_names, actual_nonames, total_score = get_score(name_pred,true_pred)
print('empty name list:', no_names/actual_nonames, '====== (counts:',no_names, '/', actual_nonames)
print('non empty name list:', nonempty/actual_nonempty, '====== (counts:',nonempty, '/', actual_nonempty)
print('avg f1:', avg_f1)
print('total score:', total_score)

empty name list: 0.9989082969432315 ====== (counts: 915 / 916
non empty name list: 0.9836065573770492 ====== (counts: 60 / 61
avg f1: 0.9494571332806625
total score: 971.9674279968395


In [27]:
print('with ckip 斷詞')
true_pred = [ast.literal_eval(i) for i in test_df['name']]
name_pred = prediction['name_pred'].to_list()
avg_f1, nonempty, actual_nonempty, no_names, actual_nonames, total_score = get_score(name_pred,true_pred)
print('empty name list:', no_names/actual_nonames, '====== (counts:',no_names, '/', actual_nonames)
print('non empty name list:', nonempty/actual_nonempty, '====== (counts:',nonempty, '/', actual_nonempty)
print('avg f1:', avg_f1)
print('total score:', total_score)

with ckip 斷詞
empty name list: 0.9865841073271414 ====== (counts: 956 / 969
non empty name list: 0.9753086419753086 ====== (counts: 158 / 162
avg f1: 0.8988189832864651
total score: 1098.0133993592622


In [30]:
print('with stanford ner')
true_pred = [ast.literal_eval(i) for i in test_df['name']]
name_pred = prediction['name_pred'].to_list()
avg_f1, nonempty, actual_nonempty, no_names, actual_nonames, total_score = get_score(name_pred,true_pred)
print('empty name list:', no_names/actual_nonames, '====== (counts:',no_names, '/', actual_nonames)
print('non empty name list:', nonempty/actual_nonempty, '====== (counts:',nonempty, '/', actual_nonempty)
print('avg f1:', avg_f1)
print('total score:', total_score)

with stanford ner
empty name list: 0.9907120743034056 ====== (counts: 960 / 969
non empty name list: 0.7777777777777778 ====== (counts: 126 / 162
avg f1: 0.761055478912622
total score: 1055.892990342991


In [79]:
sentence = '去年九合一選舉因公投造成投開票延誤爭議,前中選會主委陳英鈐因而被國民黨及民眾告發「入聯公投」、「反核食重行公告」涉瀆職'
nerrr = nlp.ner(sentence)
for n in nerrr:
    print(n[0])

去年
九合一
選舉
因
公投
造成
投
開票
延誤爭議
,
前
中選會
主委
陳英鈐
因而
被
國民黨
及
民眾
告發
「
入
聯
公投
」
、
「
反
核
食
重行
公告
」
涉瀆職


In [45]:
peter_pred.head(1)

,Unnamed: 0,pred,ans
0,0,[],[]


In [46]:
jess_pred.head(1)

,Unnamed: 0,news_ID,name_pred,name_true
0,0,9,[],[]


In [48]:
def get_name_list(x):
    ret = [i for i in (x[1:-1]).replace("'",'').split(', ')]
    if ret == ['']:
        return []
    return ret

In [59]:
peter_pred = pd.read_csv('peter_pred.csv')
jess_pred = pd.read_csv('prediction_0717.csv')

j_p = jess_pred['name_pred'].map(get_name_list)
p_p = peter_pred['pred'].map(get_name_list)
t = jess_pred['name_true'].map(get_name_list)



In [65]:
f_p = []
for i in range(len(j_p)):
    f_p.append( list( set(j_p[i]) | set( p_p[i] )))
new_prediction = pd.DataFrame()
new_prediction['pred'] = f_p
new_prediction['true'] = t


In [66]:
avg_f1, nonempty, actual_nonempty, no_names, actual_nonames, total_score = get_score(new_prediction['pred'],new_prediction['true'])
print('empty name list:', no_names/actual_nonames, '====== (counts:',no_names, '/', actual_nonames)
print('non empty name list:', nonempty/actual_nonempty, '====== (counts:',nonempty, '/', actual_nonempty)
print('avg f1:', avg_f1)
print('total score:', total_score)

empty name list: 0.9989082969432315 ====== (counts: 915 / 916
non empty name list: 1.0 ====== (counts: 61 / 61
avg f1: 0.9279085692373007
total score: 971.602422723475


In [30]:
class TestingDataset(Dataset):
    def __init__(self, MAX_LEN, tokenizer, data):
        self.data = data
        self.len = len(self.data)
        self.tokenizer = tokenizer  

    def __len__(self):
        return self.len
    
    def clean_text(self,text):
        text = text.replace('\r','')
        text = text.replace('\n','')
        return text
    
    def __getitem__(self, idx):
        MAX_LEN = 600
        content = self.data
        text = self.clean_text(content)
        inputs = self.tokenizer.encode_plus(text=text, max_length=MAX_LEN, return_tensors='pt', 
                                            pad_to_max_length = True, 
                                            return_token_type_ids = True,
                                            return_attention_mask=True)
        input_ids = inputs['input_ids'].squeeze(0)
        segments_tensor = inputs['token_type_ids'].squeeze(0)
        masks_tensor = inputs['attention_mask'].squeeze(0)
        return input_ids
    
def evaluation(MAX_LEN, tokenizer, test_loader,model):
    prediction = pd.DataFrame(columns=['news_ID','name_pred'])
    with torch.no_grad():
        ids, pred = [], []
        for data in test_loader: 
            newsid, input_ids = [t.to(device) for t in data] 
            outputs = model(input_ids)
            ans_pred = torch.sigmoid(outputs[0])
            _, ans_pred = ans_pred.max(-1)
            #print(ans_pred)
            ans_pred = list(ans_pred[0])
            #ans_index = [i for i,x in enumerate(ans_pred) if x ==2]
            ans_index = [i for i in range(len(ans_pred)) if ans_pred[i] ==1 and ans_pred[i+1]==2]
            #print(ans_index)
            text = [tokenizer.convert_ids_to_tokens(i) for i in input_ids]
            text = list(text[0])
            ans_tokens = [text[x] for i,x in enumerate(ans_index)]
            #ans_tokens = [text[ans_index[i]]+text[ans_index[i]+1] for i in range(len(ans_index)-1)]
            
            ans_tokens=[]
            a = ''
            for i in range(len(ans_index)):
                index = ans_index[i]
                if ans_pred[index].item() == 1:
                    a = text[index]
                
                    for j in range(1,4):
                        #print(text[ans_index[i]],ans_index[i],ans_pred[ans_index[i]])
                        if ans_pred[index+j].item() == 3 or index > MAX_LEN-3:
                            a += text[index+j]
                            #print('1',a)
                            break
                        else:
                            a += text[index+j]    
                ans_tokens.append(a)
                
            names, answer = [], []
            word_dict = dict()
            if ans_tokens != []:
                ws_results = ws([text])
                pos_results = pos(ws_results)
                ner_results = ner(ws_results, pos_results)
                #count word frequency
                for n in ner_results[0]:
                    if n[2] == 'PERSON' and len(n[3]) > 1 and len(n[3]) < 5:
                        names.append(n[3])
                        if n[3] in list(word_dict.keys()):
                            word_dict[n[3]] += 1 
                        else: 
                            word_dict[n[3]] = 1 
            
            names = list(set(names))
            #ckip parsing problem
            parsed_names = []
            for n in names:
                parsed_names.append([x for x in n])
            count = 0
            for n in range(len(parsed_names)):
                for m in range(len(parsed_names)):
                    if n != m:
                        n1 = parsed_names[n]
                        n2 = parsed_names[m]
                        if len(list(set(n1)&set(n2))) >= 2:
                            n1_tf = word_dict[''.join(n1)]
                            n2_tf = word_dict[''.join(n2)]
                            if n1_tf >= n2_tf:
                                if ''.join(n2) in names and len(n2) > 2 and n2_tf < 2:
                                    names.remove(''.join(n2))
                                    print('remove',''.join(n2),n1)
                                    n += 1
                            else:
                                if ''.join(n1) in names and len(n2) > 2 and n1_tf < 2:
                                    names.remove(''.join(n1))
                                    print('remove',''.join(n1),n2)
                                    n += 1
                    if n > len(parsed_names)-1:
                        break
            
            for t in ans_tokens:
                for n in names:
                    if t in n and n not in answer and len(n) < 5 and len(n) > 1:
                        if len(n) == 2 and n[1] in ['嫌','男','女', '妻']:
                            print('discard',n)
                        elif len(n) == 4 and n[0]+n[1]+n[2] in names:
                            print('discard',n)
                        else:
                            answer.append(n)
                    
            ids.append(newsid.item())
            pred.append(answer)
            print(f'news id : {newsid.item()}, names : {answer}', end = '\r')
        prediction['news_ID'] = ids
        prediction['name_pred'] = pred
    return prediction

# config = XLNetConfig.from_pretrained(PRETRAINED_MODEL_NAME)
# model = XLNetForTokenClassification(config)       
# model = model.cuda()
# checkpoint = torch.load('')
# model.load_state_dict(checkpoint)
# model = model.eval()
# MAX_LEN = 600
# BS=1
# test = TestingDataset(MAX_LEN, tokenizer, test_df)
# testloader = torch.utils.data.DataLoader(test, batch_size=BS,shuffle=False)
# prediction = evaluation(MAX_LEN, tokenizer, testloader, model)
# output = prediction['name_pred'][0]

In [29]:
names = ['王隆昌', '吳淑珍','王隆昌是']
parsed_names = []
for n in names:
    parsed_names.append([x for x in n])

word_dict = {'王隆昌':2, '吳淑珍':1, '王隆昌是':1}
for n in range(len(parsed_names)):
    for m in range(len(parsed_names)):
        if n != m:
            n1 = parsed_names[n]
            n2 = parsed_names[m]
            if len(list(set(n1)&set(n2))) >= 2:
                print(n1,n2)
                n1_tf = word_dict[''.join(n1)]
                n2_tf = word_dict[''.join(n2)]
                if n1_tf >= n2_tf:
                    if ''.join(n2) in names:
                        print('1111',n1,n2,n1_tf,n2_tf,names)
                        names.remove(''.join(n2))
                        n += 1
                else:
                    if ''.join(n1) in names:
                        print('2222',n1,n2,n1_tf,n2_tf,names)
                        names.remove(''.join(n1))
                        n += 1
        if n > len(parsed_names)-1:
            break
names

['王', '隆', '昌'] ['王', '隆', '昌', '是']
1111 ['王', '隆', '昌'] ['王', '隆', '昌', '是'] 2 1 ['王隆昌', '吳淑珍', '王隆昌是']
['王', '隆', '昌', '是'] ['王', '隆', '昌']


['王隆昌', '吳淑珍']

In [7]:
names

['王隆昌', '吳淑珍']

(33, 35, 'DATE', '本週')
(63, 66, 'PERSON', '羅雅美')
(73, 76, 'PERSON', '邱彰信')
(236, 237, 'PERSON', '邱')
(68, 71, 'ORG', '空品處')
(173, 176, 'PERSON', '羅雅美')
(10, 13, 'ORG', '國安局')
(158, 162, 'WORK_OF_ART', '中國時報')
(196, 198, 'DATE', '本週')
(222, 224, 'DATE', '日前')
(234, 235, 'PERSON', '羅')
(237, 238, 'CARDINAL', '2')
(177, 180, 'PERSON', '邱彰信')
(231, 233, 'ORG', '華航')
(92, 95, 'PERSON', '謝世謙')
(151, 154, 'ORG', '國安局')
(309, 313, 'ORG', 'TVBS')
(145, 148, 'PERSON', '張恒嘉')
(247, 250, 'PERSON', '謝世謙')
(54, 56, 'ORG', '華航')
(141, 144, 'PERSON', '吳宗憲')
(27, 30, 'ORG', '國安局')
(180, 181, 'CARDINAL', '2')


In [76]:
import spacy
from nltk import Tree

nlp = spacy.load('en')
doc = nlp(' A plate of delicious food including French fries')

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

      plate                 
  ______|______              
 |             of           
 |             |             
 |            food          
 |       ______|_______      
 |      |          including
 |      |              |     
 A      |            fries  
 |      |              |     
    delicious        French 



[None]

In [79]:
doc =  nlp('A plate of disgusting food found at a diner')
[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

      plate         
  ______|________    
 |      |      found
 |      |        |   
 |      of       at 
 |      |        |   
 |     food    diner
 |      |        |   
 A  disgusting   a  



[None]

In [31]:
import regex as re

def count_overlapping(text, search_for):
    return len(re.findall(search_for, text, overlapped=True))

count_overlapping('陳思伃','陳思千')

0

In [34]:
lista = [x for x in '陳思伃']
listb = [x for x in '陳思思']
list((set(lista) & set(listb)))

['陳', '思']

In [44]:
ws_results = ws(['今天是星期一'])
word_dict = dict()
for w in ws_results[0]:
    if w not in list(word_dict.keys()):
        word_dict[w] = 1 
    else:
        word_dict[w] += 1 
    print(w)
    
word_dict

今天
是
星期一


{'今天': 1, '是': 1, '星期一': 1}

In [30]:
names = ['郭雅雯', '賴麗團', '林勇任']
[x for x in enumerate names]

SyntaxError: invalid syntax (<ipython-input-30-b3018622b60b>, line 2)